In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ! pip uninstall -y engine

In [3]:
import torch
import numpy as np
import json
from tqdm.auto import tqdm
import random
import transformers

import os
import sys
sys.path.append('..')

from baukit import nethook
from operator import itemgetter

# from transformers import AutoModelForCausalLM, AutoTokenizer
from engine.model_manager import ModelManager


/disk/u/arnab/miniconda3/envs/engine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

MODEL_PATH="/disk/projects/engine/weights/llama-hf/llama-30b" #"facebook/galactica-6.7b"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
# MODEL_PATH = "gpt2-medium"

mt = ModelManager(
    MODEL_PATH=MODEL_PATH, 
    device_map="balanced",
    # dtype=torch.float32
)

Loading checkpoint shards: 100%|██████████| 7/7 [00:54<00:00,  7.75s/it]

model.embed_tokens.weight torch.Size([32000, 6656]) cuda:0
model.layers.0.self_attn.q_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.0.self_attn.k_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.0.self_attn.v_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.0.self_attn.o_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.0.mlp.gate_proj.weight torch.Size([17920, 6656]) cuda:0
model.layers.0.mlp.down_proj.weight torch.Size([6656, 17920]) cuda:0
model.layers.0.mlp.up_proj.weight torch.Size([17920, 6656]) cuda:0
model.layers.0.input_layernorm.weight torch.Size([6656]) cuda:0
model.layers.0.post_attention_layernorm.weight torch.Size([6656]) cuda:0
model.layers.1.self_attn.q_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.1.self_attn.k_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.1.self_attn.v_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.1.self_attn.o_proj.weight torch.Size([6656, 6656]) cuda:0
model.layers.1.mlp.gate_proj.we

In [5]:
[mt.layer_name_format.format(i) for i in range(5,10)]

['model.layers.5',
 'model.layers.6',
 'model.layers.7',
 'model.layers.8',
 'model.layers.9']

In [9]:
prompts = [
    # "Michael Jordan is a",
    "Eiffel Tower is lcoated in the city of"
]

txt, ret_dict = mt.generate(
    prompts,
    argmax_greedy= True,
    # debug = True,
    request_activations= [mt.layer_name_format.format(i) for i in range(5,10)]
)
txt

/disk/u/arnab/Projects/engine/server/notebooks/../engine/model_manager.py:204: UserWarning: The model `<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>` can't utilize `use_cache` for fast generation. Setting `use_cache = False`.
  


['<s> Eiffel Tower is lcoated in the city of Paris, France. It is one']

In [10]:
ret_dict.keys()

dict_keys(['input_tokenized', 'generated_tokens', 'activations'])

In [11]:
for k in ret_dict["activations"].keys():
    print(k, ret_dict["activations"][k].shape)

model.layers.5 (1, 19, 6656)
model.layers.6 (1, 19, 6656)
model.layers.7 (1, 19, 6656)
model.layers.8 (1, 19, 6656)
model.layers.9 (1, 19, 6656)


In [12]:
response = []

for idx in range(len(prompts)):
    result = {
        "generated_text": txt[idx],
        "input_tokenized": ret_dict["input_tokenized"][idx],
        "generated_tokens": ret_dict["generated_tokens"][idx],
        "activations": {
            k: ret_dict["activations"][k][idx] for k in ret_dict["activations"]
        }
    }

In [17]:
# result['activations']['model.layers.6'].tolist()

In [46]:
tokenized = mt.tokenizer(prompts, return_tensors="pt", padding = True).to(mt.model.device)
tokenized.pop("token_type_ids")

outputs = mt.model.generate(tokenized.input_ids.to(mt.model.device), max_new_tokens=30, top_k = 1)
print(mt.tokenizer.decode(outputs[0]))

<s> Michael Jordan is a former American professional basketball player. He is also a businessman, and principal owner and chairman of the Charlotte Hornets of the National Basketball Association (


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

model_2_name = "gpt2-medium"
model_2 = AutoModelForCausalLM.from_pretrained(
        model_2_name, low_cpu_mem_usage=True, torch_dtype=torch.float32,
)
tokenizer_2 = AutoTokenizer.from_pretrained(model_2_name)
tokenizer_2.pad_token = tokenizer_2.eos_token

In [31]:
with nethook.TraceDict(
    model, mt.layer_names
) as traces:  
    output = model(**tokenized)

In [11]:
traces[mt.layer_names[10]].output

(tensor([[[-0.2269, -0.8984, -0.8398,  ...,  0.2159,  0.0905,  0.6055],
          [-0.4167, -0.4426,  0.0248,  ...,  0.2986,  0.3564, -0.4937],
          [ 0.1370,  0.5640,  0.6880,  ..., -0.9717, -0.5552,  0.1626],
          ...,
          [ 0.0627, -0.0334, -0.3452,  ...,  0.1881,  0.0153,  0.2551],
          [ 0.6846, -0.0687, -0.1598,  ...,  0.3232, -0.0739,  0.0217],
          [ 0.5269,  0.0491, -0.7617,  ...,  0.0729,  0.1963,  0.2417]]],
        device='cuda:0', dtype=torch.float16),
 (tensor([[[[ 2.3918e-03, -3.7598e-02, -1.7670e-02,  ...,  2.2827e-01,
              2.8641e-02, -7.4341e-02],
            [ 2.5342e-01,  4.4897e-01,  1.9397e-01,  ..., -8.3691e-01,
             -9.2957e-02,  8.1494e-01],
            [-8.9233e-02, -1.0596e-01, -1.4062e-01,  ..., -1.0498e+00,
             -5.6006e-01,  9.1553e-01],
            ...,
            [-2.2070e-01,  7.3242e-02,  1.2439e-01,  ..., -4.1357e-01,
              3.9307e-01, -6.4160e-01],
            [-5.0293e-01,  1.8408e-01,  2.4

: 

In [10]:
mt = model_utils.ModelAndTokenizer(MODEL_PATH, low_cpu_mem_usage=True)

model2 = mt.model
tokenizer2 = mt.tokenizer
tokenizer2.pad_token = tokenizer2.eos_token

print(f"{MODEL_PATH} ==> device: {model2.device}, memory: {model2.get_memory_footprint()}")

gpt2-medium ==> device: cuda:0, memory: 1444458592


In [22]:
model_utils.generate_fast(
    model2, tokenizer2, 
    [input_text], 
    argmax_greedy=True,
    debug=True,
    max_new_tokens=30
)

' great'[1049] -- 10.643129348754883 [' great'(10.643) ' social'(4.839) ' wonderful'(2.623) ' platform'(2.568) ' place'(2.216) ]
' place'[1295] -- 26.1378173828125 [' place'(26.138) ' way'(14.685) ' tool'(9.537) ' platform'(8.624) ' example'(3.025) ]
' to'[284] -- 76.33644104003906 [' to'(76.336) ' for'(17.394) ','(1.438) '.'(1.046) ' where'(0.745) ]
' find'[1064] -- 13.001508712768555 [' find'(13.002) ' start'(7.52) ' share'(6.774) ' get'(5.728) ' learn'(3.726) ]
' out'[503] -- 9.319870948791504 [' out'(9.32) ' the'(3.948) ' information'(3.422) ' a'(3.387) ' great'(3.064) ]
' what'[644] -- 29.852310180664062 [' what'(29.852) ' about'(22.128) ' more'(20.307) ' how'(7.552) ' if'(2.388) ]
''s'[338] -- 28.730690002441406 [''s'(28.731) ' other'(9.443) ' people'(7.209) ' is'(4.327) ' the'(4.202) ]
' going'[1016] -- 36.31201934814453 [' going'(36.312) ' happening'(22.712) ' new'(8.658) ' trending'(4.209) ' on'(2.813) ]
' on'[319] -- 99.1840591430664 [' on'(99.184) ' down'(0.182) ' in'(0.157)

(["Facebook is a great place to find out what's going on in the world. It's a great place to find out what's going on in the world. "],
 {'past_key_values': ((tensor([[[[ 0.1553,  0.0996,  0.1718,  ..., -0.5769, -0.1995,  0.1173],
              [ 1.1678,  0.2322,  0.3601,  ...,  0.2506,  0.2609, -0.3677],
              [ 1.3599,  0.2809, -0.0502,  ...,  1.8119,  0.3128, -0.4774],
              ...,
              [ 1.1604,  0.2513, -0.1776,  ...,  2.7598,  0.2482, -0.5414],
              [-0.5904,  0.1113, -0.2870,  ...,  1.4157, -0.3826, -0.2942],
              [-1.2011,  0.2593, -0.1598,  ..., -0.9868, -0.5850, -0.6153]],
    
             [[-0.1601, -0.6617,  0.4319,  ..., -0.0878, -0.1617, -0.2335],
              [ 0.0163, -0.0235, -0.2808,  ..., -0.2326, -0.3399, -0.5331],
              [-0.0516,  0.6636, -1.5486,  ..., -0.8864, -0.7555, -1.3827],
              ...,
              [-0.2382,  0.8749, -2.0632,  ..., -1.0666, -0.7629, -1.9213],
              [-0.4626, -0.3911,  0.7434,

In [12]:
def get_zero_out_intervention(zero_out_layer):
    def zero_out_intervention(output, layer):
        if(layer == zero_out_layer):
            x = output[0]
            x = torch.zeros(output[0].shape).to(output[0].dtype).to(output[0].device)
            output = (x, output[1])
        return output
    return zero_out_intervention

In [9]:
mt.layer_names

NameError: name 'mt' is not defined

In [13]:
with nethook.TraceDict(
    model,
    mt.layer_names,
    # edit_output= get_zero_out_intervention(zero_out_layer="transformer.h.11")
) as traces:
    outputs = model(**tokenized)

In [14]:
model_utils.check_structure_tree(traces["transformer.h.11"].output)

<class 'tuple'> [2] 
├───0: <class 'torch.Tensor'>  [torch.Size([1, 3, 1024])] cuda:0
└───1: <class 'tuple'> [2] 
    ├───0: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0
    └───1: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0


In [15]:
traces["transformer.h.11"].output

(tensor([[[-0.1469,  0.7708,  1.4290,  ...,  0.5000, -0.6622, -0.2038],
          [-0.2816,  4.3047,  3.7451,  ..., -1.9087, -2.5072,  2.6908],
          [-3.5915,  2.0401,  1.5464,  ..., -4.6442, -3.9636,  2.1443]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 (tensor([[[[-0.4604, -1.2155,  0.7317,  ..., -0.0582, -0.9350,  0.2144],
            [-1.3736, -1.7998,  2.0934,  ...,  0.4302,  1.1855, -1.9837],
            [-0.7040, -1.4722,  1.6294,  ..., -1.3447, -0.5740, -1.4226]],
  
           [[ 1.1540,  0.1938, -0.2520,  ..., -1.8779, -2.2169,  2.3769],
            [-1.1719,  0.2851, -0.6967,  ..., -1.2583,  0.5322,  2.1113],
            [-0.0514, -1.0756, -1.3377,  ...,  0.5335,  0.7444,  0.7725]],
  
           [[ 1.8807, -0.3222, -1.2747,  ..., -1.1843, -1.1302,  0.2447],
            [-0.1514, -1.8124,  3.3814,  ..., -0.1750,  4.2818,  0.8564],
            [ 0.6688, -0.6131,  4.9118,  ..., -1.1441,  2.5474,  1.1113]],
  
           ...,
  
           [[ 0.5369, -0.3169,  0.3

In [16]:
isinstance(traces["transformer.h.23"].output[0], torch.Tensor)

True

In [17]:
traces["transformer.h.23"].output[0].norm().backward()

In [18]:
model_utils.check_structure_tree(traces["transformer.h.11"].output)

<class 'tuple'> [2] 
├───0: <class 'torch.Tensor'>  [torch.Size([1, 3, 1024])] cuda:0
└───1: <class 'tuple'> [2] 
    ├───0: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0
    └───1: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0


In [19]:
traces["transformer.h.11"].output

(tensor([[[-0.1469,  0.7708,  1.4290,  ...,  0.5000, -0.6622, -0.2038],
          [-0.2816,  4.3047,  3.7451,  ..., -1.9087, -2.5072,  2.6908],
          [-3.5915,  2.0401,  1.5464,  ..., -4.6442, -3.9636,  2.1443]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 (tensor([[[[-0.4604, -1.2155,  0.7317,  ..., -0.0582, -0.9350,  0.2144],
            [-1.3736, -1.7998,  2.0934,  ...,  0.4302,  1.1855, -1.9837],
            [-0.7040, -1.4722,  1.6294,  ..., -1.3447, -0.5740, -1.4226]],
  
           [[ 1.1540,  0.1938, -0.2520,  ..., -1.8779, -2.2169,  2.3769],
            [-1.1719,  0.2851, -0.6967,  ..., -1.2583,  0.5322,  2.1113],
            [-0.0514, -1.0756, -1.3377,  ...,  0.5335,  0.7444,  0.7725]],
  
           [[ 1.8807, -0.3222, -1.2747,  ..., -1.1843, -1.1302,  0.2447],
            [-0.1514, -1.8124,  3.3814,  ..., -0.1750,  4.2818,  0.8564],
            [ 0.6688, -0.6131,  4.9118,  ..., -1.1441,  2.5474,  1.1113]],
  
           ...,
  
           [[ 0.5369, -0.3169,  0.3

In [20]:
layer = nethook.get_module(model, "transformer.h.11")

In [21]:
layer.mlp.c_proj.weight.shape, layer.mlp.c_proj.weight.grad.shape

(torch.Size([4096, 1024]), torch.Size([4096, 1024]))

In [1]:
import queue

In [2]:
job_queue = queue.Queue()

job_queue.put((1,2))
job_queue.put((2,3))

In [4]:
job_queue.empty()

False

In [6]:
job_queue.get()

2

In [7]:
arr = [1,2,3,4,5]
arr[0]

1

In [8]:
arr[1:]

[2, 3, 4, 5]